# Sanfrancisco Biking - Bigquery

The goal of this shorter project is to showcase the following skills of mine:
* Connecting to Google Cloud - Bigquery API
* Limiting how much data we can fetch from the Google API
* Writing SQL queries

### 1. Import the librarys

In [1]:
import os
from google.cloud import bigquery

### 2. With the credentials file make the authorization to google.

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'application_default_credentials.json'

### 3. We are going to use the sfbiking project

In [3]:
client = bigquery.Client(project = 'sfbiking')

### 4. Create the safe_config instance, so we can add the limit to all of our future queries.

In [4]:
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)

### Inspect the data

In [5]:
df_head_query = """
                    SELECT *
                          FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                          LIMIT 5
                  """        

In [6]:
query1 = client.query(df_head_query, job_config = safe_config).result().to_dataframe()
query1

,trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code,subscriber_type
0,1235850,1540,2016-06-11 08:19:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00+00:00,San Jose Diridon Caltrain Station,2,124,15206,Customer
1,1219337,6324,2016-05-29 12:49:00+00:00,San Jose Diridon Caltrain Station,2,2016-05-29 14:34:00+00:00,San Jose Diridon Caltrain Station,2,174,55416,Customer
2,793762,115572,2015-06-04 09:22:00+00:00,San Jose Diridon Caltrain Station,2,2015-06-05 17:28:00+00:00,San Jose Diridon Caltrain Station,2,190,95391,Customer
3,453845,54120,2014-09-15 16:53:00+00:00,San Jose Diridon Caltrain Station,2,2014-09-16 07:55:00+00:00,San Jose Diridon Caltrain Station,2,127,81,Customer
4,1245113,5018,2016-06-17 20:08:00+00:00,San Jose Diridon Caltrain Station,2,2016-06-17 21:32:00+00:00,San Jose Diridon Caltrain Station,2,153,95070,Customer


### Find how many distinct records do we have.

In [7]:
query2_sql = """
                    SELECT COUNT(DISTINCT trip_id) AS Num_of_trips
                          FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  """

In [8]:
query2 = client.query(query2_sql, job_config = safe_config).result().to_dataframe()
query2.head()

,Num_of_trips
0,983648


### Find the minimum time 

In [9]:
query3_sql = """
                    SELECT MIN(DATE(start_date)) AS Earliest_time, MAX(DATE(end_date)) AS Latest_time
                          FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  """

In [10]:
query3 = client.query(query3_sql, job_config = safe_config).result().to_dataframe()
query3.head()

,Earliest_time,Latest_time
0,2013-08-29,2016-08-31


### Find the number of bikes in the bike fleet.

In [11]:
query4_sql = """
                    SELECT COUNT(DISTINCT bike_number) AS Num_of_bikes
                          FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  """

In [12]:
query4 = client.query(query4_sql, job_config = safe_config).result().to_dataframe()
query4.head()

,Num_of_bikes
0,700


### Find which Zip Code is the most common.

In [13]:
query5_sql = """
                    SELECT COUNT(*) AS Zip_code_count, zip_code
                          FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                          GROUP BY zip_code
                          ORDER BY 1 DESC
                  """

In [14]:
query5 = client.query(query5_sql, job_config = safe_config).result().to_dataframe()
query5.head()

,Zip_code_count,zip_code
0,106913,94107
1,61232,94105
2,46544,94133
3,38072,94103
4,33642,94111


### Find all the unique trips and their average,max,min trip duration as well as how many times that trip was taken.

In [15]:
 #Highest, lowest and the average time for each unique start-end station
query6_sql = """ 
                SELECT start_station_name, end_station_name, AVG(duration_sec) AS Average_duration_in_sec, MAX(duration_sec) AS Highest_Time, MIN(duration_sec) AS Lowest_Time, COUNT(*) AS Trip_count
                    FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                    GROUP BY start_station_name, end_station_name
                    ORDER BY Average_duration_in_sec
                    """

In [16]:
query6 = client.query(query6_sql, job_config = safe_config).result().to_dataframe()
query6

,start_station_name,end_station_name,Average_duration_in_sec,Highest_Time,Lowest_Time,Trip_count
0,Cyril Magnin St at Ellis St,Powell Street BART,81.000000,81,81,1
1,S. Market St at Park Ave,Adobe on Almaden,134.000000,180,88,2
2,San Salvador at 1st,5th S at E. San Salvador St,142.000000,142,142,1
3,5th S at E. San Salvador St,SJSU - San Salvador at 9th,150.000000,150,150,1
4,Beale at Market,Temporary Transbay Terminal (Howard at Beale),188.146452,14150,68,1550
...,...,...,...,...,...,...
2134,Mountain View Caltrain Station,San Jose Diridon Caltrain Station,101207.500000,189876,12539,2
2135,South Van Ness at Market,2nd at Folsom,104964.343373,17270400,590,166
2136,Santa Clara County Civic Center,California Ave Caltrain Station,169308.000000,169308,169308,1
2137,Castro Street and El Camino Real,Howard at 2nd,179212.500000,179330,179095,2


### Same as the previous query but grouped by Trip_count

In [17]:
query7_sql = """ 
                SELECT start_station_name, end_station_name, AVG(duration_sec) AS Average_duration_in_sec, MAX(duration_sec) AS Highest_Time, MIN(duration_sec) AS Lowest_Time, COUNT(*) AS Trip_count
                    FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                    GROUP BY start_station_name, end_station_name
                    ORDER BY Trip_count DESC
                    """

In [18]:
query7 = client.query(query7_sql, job_config = safe_config).result().to_dataframe()
query7

,start_station_name,end_station_name,Average_duration_in_sec,Highest_Time,Lowest_Time,Trip_count
0,Harry Bridges Plaza (Ferry Building),Embarcadero at Sansome,1188.003825,71180,219,9150
1,San Francisco Caltrain 2 (330 Townsend),Townsend at 7th,306.665961,91522,144,8508
2,2nd at Townsend,Harry Bridges Plaza (Ferry Building),581.754856,71188,294,7620
3,Harry Bridges Plaza (Ferry Building),2nd at Townsend,637.523229,37907,325,6888
4,Embarcadero at Sansome,Steuart at Market,511.009019,78123,228,6874
...,...,...,...,...,...,...
2134,South Van Ness at Market,Cyril Magnin St at Ellis St,439.000000,439,439,1
2135,Yerba Buena Center of the Arts (3rd @ Howard),Cyril Magnin St at Ellis St,359.000000,359,359,1
2136,Civic Center BART (7th at Market),Cyril Magnin St at Ellis St,303.000000,303,303,1
2137,Steuart at Market,Cyril Magnin St at Ellis St,766.000000,766,766,1


### In each year how many Subscriber and Customer trips did we have.

In [19]:
query8_sql = """ 
                SELECT SUBSTR(CAST(start_date AS STRING), 1, 4) AS Year, subscriber_type, AVG(duration_sec) AS Average_duration_in_sec, MAX(duration_sec) AS Highest_Time, MIN(duration_sec) AS Lowest_Time, COUNT(*) AS Trip_count
                    FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                    GROUP BY Year, subscriber_type
                    ORDER BY Trip_count DESC
                    """

In [20]:
query8 = client.query(query8_sql, job_config = safe_config).result().to_dataframe()
query8

,Year,subscriber_type,Average_duration_in_sec,Highest_Time,Lowest_Time,Trip_count
0,2015,Subscriber,584.849164,1852590,60,305722
1,2014,Subscriber,588.627841,716480,60,277763
2,2016,Subscriber,562.748881,85900,60,187290
3,2013,Subscriber,602.255430,619322,60,76064
4,2014,Customer,4238.843750,17270400,60,48576
5,2015,Customer,3626.959660,2137000,60,40530
6,2013,Customer,3541.320136,722236,62,24499
7,2016,Customer,2977.836925,86325,60,23204


### Query to track beginning and ending stations on October 25, 2015, for each bike

In [21]:
query9 = """
                  SELECT bike_number,
                      TIME(start_date) AS trip_time,
                      FIRST_VALUE(start_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS first_station_id,
                      LAST_VALUE(end_station_id)
                          OVER (
                               PARTITION BY bike_number
                               ORDER BY start_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                               ) AS last_station_id,
                      start_station_id,
                      end_station_id
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE DATE(start_date) = '2015-10-25' 
                  """

In [22]:
query9 = client.query(query9, job_config = safe_config).result().to_dataframe()
query9.head()


,bike_number,trip_time,first_station_id,last_station_id,start_station_id,end_station_id
0,205,14:34:00,14,14,14,14
1,530,11:18:00,82,77,82,50
2,530,12:00:00,82,77,50,73
3,530,18:52:00,82,77,73,77
4,581,12:41:00,77,73,77,60


### Query to count the (cumulative) number of trips per day

In [23]:
num_trips_query = """
                  WITH trips_by_day AS
                  (
                  SELECT DATE(start_date) AS trip_date,
                      COUNT(*) as num_trips
                  FROM `bigquery-public-data.san_francisco.bikeshare_trips`
                  WHERE EXTRACT(YEAR FROM start_date) = 2015
                  GROUP BY trip_date
                  )
                  SELECT *,
                      SUM(num_trips) 
                          OVER (
                               ORDER BY trip_date
                               ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                               ) AS cumulative_trips
                      FROM trips_by_day
                  """

In [24]:
query2 = client.query(num_trips_query, job_config = safe_config).result().to_dataframe()
query2.head()

,trip_date,num_trips,cumulative_trips
0,2015-05-28,1288,144866
1,2015-11-21,277,323385
2,2015-01-20,1213,16742
3,2015-11-22,244,323629
4,2015-12-24,124,344312
